In [2]:
# 导入数据io操作
from collections import deque
from six import next

# 调用reader.py
import readers

# Main imports for training
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


import numpy as np
import pandas as pd
import random
import math
import time
from tqdm import tqdm

from collections import defaultdict

# 评估每个轮次的训练时间
import time

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# 定义装饰器，监控运行时间
def timmer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        stop_time = time.time()
        print('函数 %s, 运行时间: %s' % (func.__name__, stop_time - start_time))
        return res
    return wrapper




In [4]:
#二维字典的添加
def addtwodimdict(thedict, key_a, key_b, val):
    if key_a in thedict:
        thedict[key_a].update({key_b: val})
    else:
        thedict.update({key_a:{key_b: val}})    

In [5]:

avg=0
num=0
@timmer
def loadData(fp="data/ml-1m/ratings.dat"):
    data = []
    user_set=set()
    item_set=set()
    
    for l in open(fp):
        temp=list(map(int,l.strip().split('::')))
        
        data.append(temp)
        user_set.add(temp[0])
        item_set.add(temp[1])
    data=list(sorted(data,key=lambda x:x[3]))

    return data,user_set,item_set

@timmer
def dealData(data):
    
    train = []
    le=len(data)
    rating_dict={}
    for user, item, rating, timestamp in data:
        addtwodimdict(rating_dict, user, item, rating)
        global avg,num
        avg+=rating
        num+=1
        train.append((user, item))

    # 处理成字典的形式，user->set(items)
    def convert_dict(data):
        data_dict = {}
        for user, item in data:
            if user not in data_dict:
                data_dict[user] = set()
            data_dict[user].add(item)
        data_dict = {k: list(data_dict[k]) for k in data_dict}
        return data_dict

    return convert_dict(train), rating_dict, 
        


In [6]:
data,user_set,item_set=loadData()


函数 loadData, 运行时间: 3.43342661857605


In [7]:
print("%d %d"%(len(user_set),len(item_set)))
u_i,rating_dict=dealData(data)

6040 3706
函数 dealData, 运行时间: 1.6007156372070312


In [8]:
# 用于复制结果的恒定种子
np.random.seed(42)

#3952  个电影 6,040个用户
u_num = 6040 
i_num = 3952  

#一次训练所选取的样本
batch_size = 1000 

# 数据的维度
dims = 10    

# 最大迭代轮次
max_epochs = 50   

# 使用设备
place_device = "/cpu:0"

In [9]:

def get_data():
    # 数据依次是用户ID、项目ID、评级、时间戳
    # 样例数据：data - 3::1196::4::978297539
    df = readers.read_file("data/ml-1m/ratings.dat", sep="::")
    
    # 获取数据的行数，待会儿要做训练和测试集的切分
    rows = len(df)
    
    # 纯粹基于整数位置的索引，根据位置进行选择
    # 实际上就是打乱一下数据的顺序 洗牌
    df = df.iloc[np.random.permutation(rows)].reset_index(drop=True)
    
    # 90%用作训练，10%用作测试
    split_index = int(rows * 0.9)
    
    # Use indices to separate the data
    df_train = df[0:split_index]
    df_test = df[split_index:].reset_index(drop=True)
    
    return df_train, df_test

def clip(x):
    return np.clip(x, 1.0, 5.0)

In [10]:
def model(user_batch, item_batch, user_num, item_num, dim=100, device="/cpu:0"):
    with tf.device("/cpu:0"):
        # 变量域
        with tf.variable_scope('lsi',reuse=tf.AUTO_REUSE):
            # 全局偏置变量
            # get_variable:在名称前面加上当前变量作用域并执行重用检查
            bias_global = tf.get_variable("bias_global",shape=[])
            
            # 用户的偏好
            w_bias_user = tf.get_variable("embd_bias_user", shape=[user_num])
            # 电影的偏好
            w_bias_item = tf.get_variable("embd_bias_item", shape=[item_num])
            
            # 用户和电影一个batch的偏好
            bias_user = tf.nn.embedding_lookup(w_bias_user, user_batch, name="bias_user")
            bias_item = tf.nn.embedding_lookup(w_bias_item, item_batch, name="bias_item")
            
            # 用户和电影的权重
            w_user = tf.get_variable("embd_user", shape=[user_num, dim],
                                     initializer=tf.truncated_normal_initializer(stddev=0.02))
            w_item = tf.get_variable("embd_item", shape=[item_num, dim],
                                     initializer=tf.truncated_normal_initializer(stddev=0.02))
            
            # 给定批处理的用户和项的权重嵌入
            # 用户和电影一个batch的权重
            embd_user = tf.nn.embedding_lookup(w_user, user_batch, name="embedding_user")
            embd_item = tf.nn.embedding_lookup(w_item, item_batch, name="embedding_item")
    
    with tf.device(device):
        # 计算张量各维度元素和
        infer = tf.reduce_sum(tf.multiply(embd_user, embd_item), 1)
        
        infer = tf.add(infer, bias_global)
        infer = tf.add(infer, bias_user)
        infer = tf.add(infer, bias_item, name="svd_inference")
        
        # 加上L2的正则化项
        # l2_loss: 计算一个张量的L2范数的一半
        # regularizer：正则化项
        regularizer = tf.add(tf.nn.l2_loss(embd_user), tf.nn.l2_loss(embd_item), 
                             name="svd_regularizer")

    # 返回我们预测的结果和正则化项
    return infer, regularizer

In [11]:
def loss(infer, regularizer, rate_batch, learning_rate=0.002, reg=0.01, device="/cpu:0"):
    with tf.device(device):
        # 使用L2 loss算出预测值到实际值的距离 
        # infer  预测值    rate_batch 实际值
        cost_l2 = tf.nn.l2_loss(tf.subtract(infer, rate_batch))
        
        # 惩罚的方式----L2
        penalty = tf.constant(reg, dtype=tf.float32, shape=[], name="l2")
        
        # 损失函数 = 数据损失（data loss） + 正则化损失（正则化项 * L2惩罚方式）
        cost = tf.add(cost_l2, tf.multiply(regularizer, penalty))
        
        # 训练 使用梯度下降
        train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    return cost, train_op

In [12]:
# 从评级文件读取数据以构建 tensorflow 模型
df_train, df_test = get_data()

samples_per_batch = len(df_train) // batch_size
print("Number of train samples %d, test samples %d, samples per batch %d" % 
      (len(df_train), len(df_test), samples_per_batch))

Number of train samples 900188, test samples 100021, samples per batch 900


In [13]:
# 查看前5个用户值
se=set()
for it in df_train["item"]:
    se.add(it)
for it in df_test["item"]:
    se.add(it)
print(len(se))
print(df_test["user"].head())

3706
0    1696
1    5448
2    2242
3    5629
4     423
Name: user, dtype: int32


In [14]:
# 使用shuffle迭代器生成随机批次，用于训练
iter_train = readers.ShuffleIterator([df_train["user"],
                                     df_train["item"],
                                     df_train["rate"]],
                                     batch_size=batch_size)

# 按顺序生成一个epoch的batch用于测试
iter_test = readers.OneEpochIterator([df_test["user"],
                                     df_test["item"],
                                     df_test["rate"]],
                                     batch_size=-1)

# 创建占位符
user_batch = tf.placeholder(tf.int32, shape=[None], name="id_user")
item_batch = tf.placeholder(tf.int32, shape=[None], name="id_item")
rate_batch = tf.placeholder(tf.float32, shape=[None])

infer, regularizer = model(user_batch, item_batch, user_num=u_num, item_num=i_num, dim=dims, device=place_device)
_, train_op = loss(infer, regularizer, rate_batch, learning_rate=0.0010, reg=0.05, device=place_device)

In [15]:
UI=[[0 for j in range(i_num)] for i in range(u_num)]


In [16]:
saver = tf.train.Saver()
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    print("%s\t%s\t%s\t%s" % ("Epoch", "Train Error", "Val Error", "Elapsed Time"))
    errors = deque(maxlen=samples_per_batch)
    start = time.time()
   
    for i in range(max_epochs * samples_per_batch):
        users, items, rates = next(iter_train)
  
        _, pred_batch = sess.run([train_op, infer], feed_dict={user_batch: users,
                                                               item_batch: items,
                                                               rate_batch: rates})
        pred_batch = clip(pred_batch)
        errors.append(np.power(pred_batch - rates, 2))
        
        if i % samples_per_batch == 0:
            train_err = np.sqrt(np.mean(errors))
            test_err2 = np.array([])
            for users, items, rates in iter_test:
                pred_batch = sess.run(infer, feed_dict={user_batch: users,
                                                        item_batch: items})
                pred_batch = clip(pred_batch)
                #print(pred_batch)
                test_err2 = np.append(test_err2, np.power(pred_batch - rates, 2))

                
            
            end = time.time()
            
            print("%02d\t%.3f\t\t%.3f\t\t%.3f secs" % (i // samples_per_batch, train_err, np.sqrt(np.mean(test_err2)), end - start))
            start = end
            
    for var in tf.global_variables():
        print(var.shape)
        f_name=var.name.split('/')[-1].split(':')[0]  
        data_numpy=var.eval()
        
        if f_name=="bias_global":continue
        f_name=f_name+".csv"
        print(f_name)
        np.savetxt(f_name, data_numpy, delimiter=",")

                
    saver.save(sess, './save/')

Epoch	Train Error	Val Error	Elapsed Time
00	2.823		1.116		0.230 secs
01	1.048		1.003		0.803 secs
02	0.982		0.967		0.764 secs
03	0.955		0.950		0.760 secs
04	0.941		0.939		0.808 secs
05	0.932		0.932		0.820 secs
06	0.926		0.928		0.765 secs
07	0.922		0.924		0.826 secs
08	0.919		0.922		0.821 secs
09	0.915		0.920		0.914 secs
10	0.913		0.918		0.725 secs
11	0.911		0.917		0.723 secs
12	0.909		0.919		0.718 secs
13	0.908		0.915		0.867 secs
14	0.909		0.913		0.841 secs
15	0.907		0.915		0.783 secs
16	0.907		0.912		0.779 secs
17	0.905		0.911		0.797 secs
18	0.905		0.911		0.888 secs
19	0.905		0.911		0.871 secs
20	0.903		0.912		0.881 secs
21	0.903		0.910		0.864 secs
22	0.902		0.910		0.738 secs
23	0.902		0.910		0.841 secs
24	0.902		0.909		0.802 secs
25	0.903		0.909		0.915 secs
26	0.901		0.909		0.775 secs
27	0.903		0.909		0.769 secs
28	0.901		0.909		0.728 secs
29	0.901		0.908		0.833 secs
30	0.901		0.909		0.807 secs
31	0.901		0.908		0.778 secs
32	0.900		0.908		0.803 secs
33	0.900		0.908		0.797 secs
34	0.90

In [17]:
def loadDataSet(fileName,n,m):
    f = open(fileName)
    rownum=0
    train_X_matrix = np.empty((n,m),np.float64)
    for line in f.readlines():
        train_X_matrix[rownum] = np.asarray(line.strip('\n ').split(','), dtype=float)
        rownum += 1
    return train_X_matrix

In [18]:
P=loadDataSet("embd_user.csv",u_num,10).T
Q=loadDataSet("embd_item.csv",i_num,10)
u_ba=loadDataSet("embd_bias_item.csv",u_num,1)
i_ba=loadDataSet("embd_bias_item.csv",i_num,1)
print(P.shape)
print(Q.shape)

(10, 6040)
(3952, 10)


In [19]:
UI=np.dot(Q,P)
for i in range(0,i_num):
    for j in range(0,u_num):
        
        if i+1 in rating_dict:
            if j+1 in rating_dict[i+1]:
                UI[i][j]=rating_dict[i+1][j+1]
                continue
                
        UI[i][j]+=u_ba[j]+i_ba[i]+avg/num
    
np.savetxt("UI.csv", UI, delimiter=",")

In [24]:

from Bio.Cluster import * 
#用k-means 或 k-median进行聚类，K为聚类数
def KCluster(data, K=10):
    clusterid, error, nfound = kcluster (data,npass=100,nclusters=K,mask=None, weight=None,  dist='c', initialid=None)
    cdata, cmask = clustercentroids(data, mask=None, clusterid=clusterid)
    #silhouette_avg = silhouette_score(data, clusterid)
    return clusterid,cdata

In [25]:
UI=np.clip(UI, 0.0, 5.0) 

In [26]:

clusterid,cdata=KCluster(UI, 10)
best_clusterid=clusterid

In [27]:
in_d=[0 for i in range(0,i_num+1)]
out_d=[0 for i in range(0,i_num+1)]

for  u in u_i:
    pre=-1
    for it in u_i[u]:
        it-=1
        if pre==-1:
            in_d[it]+=1
            pre=it
            continue
        
        if best_clusterid[it]!=best_clusterid[pre]:

            out_d[pre]+=1
            in_d[it]+=1
        pre=it
        
    
        
   
    
    

In [28]:
test=pd.DataFrame(data=best_clusterid)
test.to_csv('best_clusterid.csv')
test=pd.DataFrame(data=cdata)
test.to_csv('cdata.csv')
test=pd.DataFrame(data=in_d)
test.to_csv('in_d.csv')
test=pd.DataFrame(data=out_d)
test.to_csv('out_d.csv')

In [29]:
print(cdata)

[[4.35100338 3.10462087 2.70600504 ... 2.71827485 2.7180628  2.71790085]
 [4.72112812 4.31051225 4.13763183 ... 4.49921034 4.49886763 4.49974214]
 [4.32173394 3.41650357 3.24252654 ... 3.58340725 3.5834713  3.58335983]
 ...
 [4.0375928  2.98763645 2.81762606 ... 3.03420124 3.0343395  3.03406476]
 [3.92815612 3.33833789 3.30075689 ... 4.20416056 4.20403103 4.20442974]
 [4.28114269 3.49120515 3.37108617 ... 3.86010284 3.86012074 3.86016982]]
